In [1]:
# Install libraries

!pip install -q -U bitsandbytes accelerate transformers datasets trl peft

In [2]:
# Hugging Face Login

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")

import huggingface_hub
huggingface_hub.login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [44]:
# Import Modules

import warnings
warnings.filterwarnings("ignore")

import torch
import pandas as pd

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [45]:
# Load Dataset

from datasets import load_dataset

dataset = load_dataset("bky373/spring-docs")

README.md:   0%|          | 0.00/205 [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6205 [00:00<?, ? examples/s]

In [46]:
# 데이터셋 탐색 및 예시

dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 6205
    })
})

In [47]:
dataset['train'][750]

{'question': '스프링 부트에서 ThreadPoolTaskExecutor의 스레드 풀 크기와 큐 용량을 어떻게 설정할 수 있나요?',
 'answer': 'ThreadPoolTaskExecutor의 스레드 풀 크기와 큐 용량은 spring.task.execution 네임스페이스를 사용하여 설정할 수 있습니다. 예를 들어, spring.task.execution.pool.max-size=16, spring.task.execution.pool.queue-capacity=100와 같이 설정하면 스레드 풀 크기와 큐 용량을 각각 16개와 100개로 설정할 수 있습니다. (출처: https://docs.spring.io/spring-boot/reference/features/task-execution-and-scheduling.html)'}

In [48]:
# Initialize Base Model

base_model = "google/gemma-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4비트 양자화를 활성화, 모델을 메모리 효율적으로 로드
    bnb_4bit_quant_type="nf4",  # 4비트 양자화에서 사용할 방식(Normal Float 4), 더 나은 표현력 제공
    bnb_4bit_compute_dtype=torch.float16  # 양자화된 값의 계산에 사용할 데이터 타입, FP16으로 설정하여 메모리 절약
)

model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", quantization_config=bnb_config)

model.config.use_cache = False

# model.config.pretraining_tp = 1

max_seq_length = 1024
tokenizer = AutoTokenizer.from_pretrained(base_model, max_seq_length=max_seq_length)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# Build QA Dataset

def build_qa_pairs(data):
    pairs = []
    for i in range(len(data)):
        pairs.append(f"question:{data['question'][i]}\nanswer:{data['answer'][i]}")
    return pairs

qa_dataset = build_qa_pairs(dataset['train'])
qa_dataset[750]

'question:스프링 부트에서 ThreadPoolTaskExecutor의 스레드 풀 크기와 큐 용량을 어떻게 설정할 수 있나요?\nanswer:ThreadPoolTaskExecutor의 스레드 풀 크기와 큐 용량은 spring.task.execution 네임스페이스를 사용하여 설정할 수 있습니다. 예를 들어, spring.task.execution.pool.max-size=16, spring.task.execution.pool.queue-capacity=100와 같이 설정하면 스레드 풀 크기와 큐 용량을 각각 16개와 100개로 설정할 수 있습니다. (출처: https://docs.spring.io/spring-boot/reference/features/task-execution-and-scheduling.html)'

In [42]:
train_data = (pd.DataFrame(qa_dataset, columns=["text"])
              .sample(frac=1, random_state=5)
              .drop_duplicates()
             )
train_data = Dataset.from_pandas(train_data)

In [ ]:
output_dir = "gemma2-2b_spring-assistant-ft"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1, # 과적합을 방지하고 모델의 일반화 능력을 향상시키기 위함
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    gradient_checkpointing=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=100,
    learning_rate=1e-4,
    weight_decay=0.01,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=1500,
    warmup_ratio=0.1,
    group_by_length=False,
    evaluation_strategy='steps',
    do_eval=True,
    eval_steps = 100,
    eval_accumulation_steps=1,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
)


In [18]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    args=training_arguments,
    packing=False,
)

Map:   0%|          | 0/6205 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [19]:
trainer.train()

Step,Training Loss
100,1.768400
200,1.447300
300,1.372900
400,1.312400
500,1.271000
600,1.217800
700,1.182400
800,1.104800
900,0.872900
1000,0.843100


TrainOutput(global_step=1000, training_loss=1.239297004699707, metrics={'train_runtime': 4588.517, 'train_samples_per_second': 1.743, 'train_steps_per_second': 0.218, 'total_flos': 1.3536349998587904e+16, 'train_loss': 1.239297004699707, 'epoch': 1.2892828364222402})

In [20]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)

('gemma2-2b_spring-assistant-ft/tokenizer_config.json',
 'gemma2-2b_spring-assistant-ft/special_tokens_map.json',
 'gemma2-2b_spring-assistant-ft/tokenizer.model',
 'gemma2-2b_spring-assistant-ft/added_tokens.json',
 'gemma2-2b_spring-assistant-ft/tokenizer.json')

In [21]:
import gc

del [model, tokenizer, peft_config, trainer, train_data, bnb_config, training_arguments]
del [TrainingArguments, SFTTrainer, LoraConfig, BitsAndBytesConfig]

for _ in range(10):
    torch.cuda.empty_cache()
    gc.collect()

In [26]:
from peft import AutoPeftModelForCausalLM
 
tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoPeftModelForCausalLM.from_pretrained(
     output_dir,
     low_cpu_mem_usage=True,
     device_map="auto",
     torch_dtype=torch.float16
)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

merged_model = model.merge_and_unload()
merged_model_name = "./gemma2_merged_spring-assistant"
merged_model.save_pretrained(merged_model_name, safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained(merged_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('./gemma2_merged_spring-assistant/tokenizer_config.json',
 './gemma2_merged_spring-assistant/special_tokens_map.json',
 './gemma2_merged_spring-assistant/tokenizer.model',
 './gemma2_merged_spring-assistant/added_tokens.json',
 './gemma2_merged_spring-assistant/tokenizer.json')

In [27]:
import gc

del [model, tokenizer, merged_model, AutoPeftModelForCausalLM]

for _ in range(10):
    torch.cuda.empty_cache()
    gc.collect()

In [29]:
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    merged_model_name,
    device_map="auto",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
# model.config.pretraining_tp = 1

max_seq_length = 1024
tokenizer = AutoTokenizer.from_pretrained(merged_model_name, max_seq_length=max_seq_length)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [39]:
def generate_ai_response(user_input):
    messages = [
        {
            "role": "user",
            "content": user_input
        }
    ]
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    result = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.8)
    
    generated_text = result[0]['generated_text']
    return generated_text.strip()

In [48]:
generate_ai_response("Spring AI에서 AWS Bedrock Cohere chat model을 사용하여 요청 특정 런타임 옵션을 어떻게 사용할 수 있나요?")

'Spring AI에서 AWS Bedrock Cohere chat model을 사용하여 요청 특정 런타임 옵션을 사용하려면, AWS Bedrock Cohere Chat Model 클라이언트 팩토리를 구성하면 됩니다. Spring AI 문서에서 제공하는 예제를 참조하여 제공되는 옵션을 사용할 수 있습니다. 이 예제에서는 최적화된 모델을 사용하는 방법을 보여줍니다. 이 예제는 텍스트 기반 텍스트, 텍스트 기반 텍스트와 이미지, 텍스트 기반 텍스트와 비디오, 텍스트 기반 텍스트와 audio 또는 video, 텍스트 기반 텍스트와 이미지, 텍스트 기반 텍스트와 audio, 텍스트 기반 텍스트와 video와 함께 이미지를 포함하는 텍스트입니다. (출처: https://docs.spring.io/spring-ai/reference/1.0-SNAPSHOT/api/cloud-aws/bedrock-chat-model.html) 이 예제는 AWS Bedrock Cohere Chat Model을 사용하여 요청 특정 런타임 옵션을 사용하는 방법을 보여줍니다. 또한, 이 예제는 텍스트 기반 텍스트와 이미지, 텍스트 기반 텍스트와 audio 또는 video, 텍스트 기반 텍스트와 이미지, 텍스트 기반 텍스트와 audio, 텍스트 기반 텍스트와 video와 함께 이미지를 포함하는 텍스트입니다. (출처: https://docs.spring.io/spring-ai/reference/1.0-SNAPSHOT/api/cloud-aws/bedrock-chat-model.html) 이 예제는 AWS Bedrock Cohere Chat Model을 사용하여 요청 특정 런타임 옵션을 사용하는 방법을 보여줍니다. 또한, 이 예제는 텍스트 기반 텍스트와 이미지, 텍스트 기반 텍스트와 audio 또는 video, 텍스트 기반 텍스트와 이미지, 텍스트 기반 텍스트와 audio, 텍스트 기반 텍스트와 video와 함께 이미지를 포함하는 텍스트입니다. (출처: https://docs.spring.io/spring

In [49]:
generate_ai_response("스프링 인증 서버가 지원하는 토큰 형식은 무엇인가요?")

'스프링 인증 서버는 JSON Web Token (JWT), JSON Web Signature (JWS), OAuth 2.0 Token Exchange, OAuth 2.0 Authorization Code Grant, OAuth 2.0 Client Credentials Grant, OAuth 2.0 Password Credentials Grant, OAuth 2.0 Client Credentials Refresh Grant, OAuth 2.0 Refresh Token Grant, OAuth 2.0 Token Exchange Token Request, OAuth 2.0 Token Exchange Token Refresh Token Request, OAuth 2.0 Token Exchange Token Refresh Token Refresh Token Request, OAuth 2.0 Token Exchange Token Refresh Token Refresh Token Refresh Token Request, OAuth 2.0 Bearer Token Request, OAuth 2.0 Refresh Token Request, OAuth 2.0 Access Code Request, OAuth 2.0 Bearer Assertion Request, OAuth 2.0 Client Credentials Request, OAuth 2.0 Client Credentials Refresh Request, OAuth 2.0 Client Credentials Refresh Refresh Request, OAuth 2.0 Bearer Token Request, OAuth 2.0 Token Request 및 OAuth 2.0 Refresh Token Request를 지원하며, 각각 JWT, JWS, OAuth 2.0 Client Credentials Grant, OAuth 2.0 Refresh Token Request, OAuth 2.0 Client Credentials Refresh

In [51]:
generate_ai_response("스프링 부트에서 메시징 프로토콜을 사용하려면 어떻게 해야 하나요?")

"스프링 부트는 Spring Cloud Gateway에서 메시징 프로토콜을 사용할 수 있도록 제공합니다. 먼저, 'configureExternalTransportGateway' 메소드를 사용하여 HTTP 또는 TCP 메시징 프레임워크를 구성해야 합니다. 그런 다음, 'configureExternalTransportGateway' 메소드를 사용하여 메시징 프로토콜에 대한 구성을 추가해야 합니다. 이는 기본적으로 HTTP를 사용하며, TCP를 사용하려면 'tcp' 속성을 true로 설정해야 합니다. 이를 통해 메시징 프로토콜을 사용할 수 있습니다. (출처: https://docs.spring.io/spring-cloud-gateway/reference/html/spring-cloud-gateway-server-mvc.html) 자세한 내용은 다음 섹션에서 확인할 수 있습니다: https://docs.spring.io/spring-cloud-gateway/reference/html/spring-cloud-gateway-server-mvc.html#spring-cloud-gateway-server-mvc-supported-protocols-http-tcp-udp-tcp-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls-ssl-tls.html에서 확인할 수 있습니다. (출처: https://docs.spring.io/spring-cloud-gateway/reference/html/spring-cloud-gateway-server-mvc.html) (출처: https://docs.spring.io/spring-cloud-gateway/reference/html/spring-cloud-gateway-server-mvc.html) (출처: https://docs.spring.io/spring-clo